# Bayes Estimation

## References

- https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote05.html

## Naive Bayes

\begin{align}
    P(Y=y | X=x) &= \frac{P(X=x | Y=y) P(Y=y)}{P(X=x)}\\
    \\
    &\text{Where } \\
    P(X=x | Y=y) &= \prod_{\alpha=1}^{d} P([X]_\alpha = x_\alpha| Y = y)
\end{align}


- Naively assumes that all the features used are independently distrubuted variables given the label Y.
- for example given that there is an email where all the words are independent given the label spam/ham.

## Bayes Classifier

\begin{align*}
    h(\vec{x}) &= {argmax\atop{y}} \frac{P(\vec{x} | y) P(y)}{z}\\
    \\
    &= {argmax\atop{y}} P(y) \prod_{\alpha} P([\vec{X}]_\alpha | y)\\
    \\
    &= {argmax\atop{y}} ( log(P(y) + \sum_\alpha log P([\vec{X}]_\alpha | y))
\end{align*}


P.S. - In computer science we dont prefer multiplying probabilities due to muliple reasons(see reference section). Hence we take log and convert multiplication to addition.